In [6]:
import os
import re
import json
from pprint import pprint

# Load and prepare the data

In [3]:
all_files = os.listdir("chunk_out")

In [10]:
a_file = all_files[0]
a_file
folder = "chunk_out"

In [8]:
# fetch everything before the first occurance of a opening curly brace
# to handle HTTP response headers present in the files
pattern = re.compile(r"[^{]*")

In [34]:
# clean up the data files for JSON traversion
documentation = ""
response_list = list()

for a_file in all_files:
    # fetch the current file
    with open("{0}/{1}".format(folder, a_file)) as f:
        file_string = f.read()
        header = re.search(pattern, file_string)
        documentation += "\t{0}\n\n{1}".format(a_file, header.group(0))
        # delete the HTML response header from the JSON
        file_string = file_string.replace(header.group(0), "")
        # load the JSON into a python dict
        response = json.loads(file_string)
        response_list.append(response)

In [37]:
#print(documentation)

In [36]:
# saving the documentation headers, just because
with open("doc_headers.txt", "w") as f:
    f.write(documentation)

# Traversing the JSON valleys

In [ ]:
def id_generator(dict):
      for k, v in dict.items():
            yield k
#            if k == "lemma":
#                 yield v
#            elif isinstance(v, dict):
#                 for id_val in id_generator(v):
#                       yield id_val

In [42]:
def item_generator(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                for child_val in item_generator(v, lookup_key):
                    yield child_val
    elif isinstance(json_input, list):
        for item in json_input:
            for item_val in item_generator(item, lookup_key):
                yield item_val

In [49]:
#with open("{0}/try.json".format(folder), "r") as f:
#    res = json.load(f)    
count = 0
form_num = 0
for _ in item_generator(response_list[0], "form"):
    form_num += 1
#    count += 1
#    if count >= 5:
#        break

In [54]:
list_list = []


for _ in item_generator(response_list[0], "analysis_list"):
    list_list.append(_[0])

In [65]:
focus_list = list()
messy_list = list()

for l in list_list:
    # take only those forward that are actually single words
    if l["original_form"].find(" ") == -1:
        focus_list.append(l)
    # put the rest into another list (for checkup)
    else:
        messy_list.append(l)    

In [66]:
len(messy_list), len(focus_list)

(272, 11700)

In [62]:
import pandas as pd

In [69]:
df = pd.DataFrame(focus_list)

In [70]:
df.head()

,check_info,lemma,original_form,remission,sense_id_list,tag,variety_dictionary
0,NaN,primero,primer,NaN,NaN,MDMSOT7,NaN
1,NaN,primero,primera,NaN,NaN,MPFSON7,NaN
2,NaN,primero,primeras,NaN,NaN,MDFPON7,NaN
3,NaN,primero,primero,NaN,NaN,ENMPN7,NaN
4,NaN,primero,primero,NaN,NaN,GE---M---,NaN


In [91]:
nef = df[['lemma', 'original_form', 'tag']]
nef.head()

,lemma,original_form,tag
0,primero,primer,MDMSOT7
1,primero,primera,MPFSON7
2,primero,primeras,MDFPON7
3,primero,primero,ENMPN7
4,primero,primero,GE---M---


In [80]:
fdf = df.drop(["check_info", "remission", "sense_id_list", "variety_dictionary"], axis=1)
fdf[7000:7030]

,lemma,original_form,tag
7000,aborregar,aborreguéis,VS-P2PSAPL-N-
7001,aborregar,aborreguéis,GV-P2----
7002,aborregar,aborreguemos,VS-P1PSAPL-N-
7003,aborregar,aborreguemos,GV-P1----
7004,aborregar,aborreguen,VS-P3PSAPL-N-
7005,aborregar,aborreguen,GV-P3----
7006,aborregar,aborregues,VS-S2PSAPL-N-
7007,aborregar,aborregues,GV-S2----
7008,abortamiento,abortamientos,NCMP-NYN-
7009,abortamiento,abortamientos,GNMP3S---
